<a href="https://colab.research.google.com/github/zhuhel/RPC_TOF_NN/blob/main/V0830_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
########################################################################################
## PACKAGE IMPORTING AND GLOBAL VARIABLE DEFINITION ########
########################################################################################

# from 721, set bias=False for LSTM and MLP; LSMT layer =1 and no dropout # one layer LSTM may be enough
# V0.7  Update: Use large hidden size; MSE as loss function; Based on V0.60
# V0.6  Update: Type 3 has better performance than type 4
# V0.51 Update: to run on the oscilloscope data
# V0.40 Update: to come different networks
# V0.31 Update: tof changed to tensor format, which is a grammar improvement instead of bug fix
# V0.32 Update: Use L2 regularization instead of drapout

from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable
from torch import optim
import torch    # for setting seed to make it reproducible
import torch.nn as nn
import matplotlib.pyplot as plt  # for plotting
import matplotlib.mlab as mlab  # for plotting
import struct  # for binary file reading
import numpy as np
import time  # for time recording
import math  # for sqrt
from pathlib import Path  # for the size of the binary file
from scipy.stats import norm  # for gaussian fit
import sys  # for parameter receiving
import os  # for mkdir
from progressbar import *

os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
time_start = time.time()
time_epoch = time.time()

SEED=5  # reproducible
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
np.random.seed(SEED)
torch.backends.cudnn.deterministic = True
torch.manual_seed(SEED)
SAMPLE_EPOCH = 1  # test every xx epochs to record mu, timeresolution, and loss
EPOCHS = 100

MODEL_VERSION = '830'
DATA_VERSION = 'Aug-200_900_6'
WAVE_LENGTH = 160
NCHANNEL = 2

LR = 0.0001 + 0.0009*np.random.rand()
LRD = 0.9 + 0.1*np.random.rand()
HIDDEN_SIZE = 60 + int(10*np.random.rand())
NUM_LSTM_LAYERS = 4 #+int(2*np.random.rand())
BATCH_SIZE = 256
DROP_OUT = 0.0
WEIGHT_DECAY = 0.0004 + 0.0004*np.random.rand()  # lambda of L2 regularization

In [ ]:
########################################################################################
## CLASS AND FUNCTIION DEFINITION ########
########################################################################################

class Waveforms(Dataset):
    def __init__(self, root_dir, testRate = 0.3, train=True):
        self.root_dir = root_dir
        self.train = train
        self.file = open(root_dir, 'rb')
        self.waveLength = WAVE_LENGTH
        self.NChannel = NCHANNEL
        self.NEvents = Path(self.root_dir).stat().st_size / \
            (self.waveLength*self.NChannel+1)/4
        print('NEvents = ', self.NEvents)
        self.testRaio = testRate

    def __len__(self):
        if self.train:
            return int(self.NEvents*(1.0-self.testRaio))
        else:
            return int(self.NEvents*self.testRaio)

    def __getitem__(self, index):
        if self.train:
            self.file.seek((index+int(self.NEvents*self.testRaio))
                           * (self.NChannel*self.waveLength+1)*4, 0)
        else:
            self.file.seek((index)*(self.NChannel*self.waveLength+1)*4, 0)
        data = self.file.read(self.waveLength*self.NChannel*4)
        wave = struct.unpack(str(self.NChannel*self.waveLength)+'f', data)
        wave_tensor = torch.tensor(wave, dtype=torch.float32).reshape(
            self.NChannel, self.waveLength)
        data = self.file.read(4)
        tof = struct.unpack('f', data)
        tof_tensor = torch.tensor(tof, dtype=torch.float32)
        sample = {'waveform': wave_tensor, 'tof': tof_tensor}
        return sample


class MLP(nn.Module):
    def __init__(self, input_size, common_size):
        super(MLP, self).__init__()
        self.linear = nn.Sequential(
            nn.Linear(input_size, input_size // 2),
            nn.ReLU(inplace=True),
            nn.Linear(input_size // 2, input_size // 4),
            nn.ReLU(inplace=True),
            nn.Linear(input_size // 4, common_size)
        )

    def forward(self, x):
        out = self.linear(x)
        return out

class ComNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=WAVE_LENGTH,
            hidden_size=HIDDEN_SIZE,
            num_layers=NUM_LSTM_LAYERS,
            batch_first=True,
            bias=False,
            dropout=DROP_OUT
        )
        self.out1 = nn.Linear(HIDDEN_SIZE, 1, bias=False)
        # self.magnifier1 = MLP(NCHANNEL*WAVE_LENGTH, WAVE_LENGTH) # N to one
        # self.magnifier2 = MLP(NCHANNEL*WAVE_LENGTH, WAVE_LENGTH) # N to one


    def forward(self, x):
        x.dim
        # # print('x:\t', x.size())
        # extra1 = self.magnifier1(x.view(x.size(0),NCHANNEL*WAVE_LENGTH))
        # extra1 = extra1.unsqueeze(2)
        # extra2 = self.magnifier2(x.view(x.size(0),NCHANNEL*WAVE_LENGTH))
        # extra2 = extra2.unsqueeze(2)
        # # print('extra:\t', extra.size())
        # x = torch.cat((x,extra1), 2)
        # x = torch.cat((x,extra2), 2)
        # print('cat:\t', x.size())

        lstm_out, (h_n, h_c) = self.lstm(x, None)
        # lstm_out[:, -1, :] is the last lines of the input batches, since it's a many to one IO structure.
        out = (self.out1(lstm_out[:, -1, :]))
        return out


def testDataset(network, dataVersion, outputPrefix, input_prefix = "./Codes/V0.70/Data/"):
    inputFileName = input_prefix+dataVersion+".bin"
    testWaveformDataset = Waveforms(inputFileName, testRate = 1.0, train=False)
    testData = DataLoader(testWaveformDataset,
                          batch_size=BATCH_SIZE, shuffle=False)

    residualTofList = []
    predictedTofList = []
    labelTofList = []
    print("Test Data: " + dataVersion)
    mu=sigma=0
    with torch.no_grad( ):
        for index, batch_data in enumerate(testData):
            input = Variable(batch_data['waveform']).cuda()
            b_y = Variable(batch_data['tof']).cuda().squeeze()
            output = comNN(input).squeeze()
            residualTofList.extend(
                (output-b_y).cpu().detach().numpy().tolist())
            predictedTofList.extend(output.cpu().detach().numpy().tolist())
            labelTofList.extend(b_y.cpu().detach().numpy().tolist())

    (mu, sigma) = norm.fit(residualTofList)
    n, bins, patches = plt.hist(residualTofList, 50, range=(-2, 2), density=1)
    # add a 'best fit' line
    y = norm.pdf(bins, mu, sigma)
    l = plt.plot(bins, y, 'r--', linewidth=2)
    plt.xlabel('Tof residual [ns]')
    plt.title(
        r'$\mathrm{Tof\ residual:}\ \mu=%.3f ns,\ \sigma/\sqrt{2}=%.3f ns$' % (mu, sigma/math.sqrt(2)))
    plt.grid(True)
    plt.savefig(outputPath + outputPrefix +
                "Test_" + dataVersion + ".png")
    plt.clf()


def my_mse_loss(x, y, n): 
    return torch.mean(torch.pow(torch.abs(x - y), n))


In [ ]:
########################################################################################
## INPUT/OUTPUT PATH SETTING ########
########################################################################################

workspace = '/content/gdrive/My Drive/RPC_TOF/'
fileName = workspace+"/data/"+DATA_VERSION+".bin"
outputPath = workspace+DATA_VERSION+"_"+MODEL_VERSION+"_Aug20th_v1/"
if os.path.exists(outputPath) == False:
    os.mkdir(outputPath)
outputPrefix = DATA_VERSION+"_SEED"+str(SEED).zfill(4) + "_E"+str(EPOCHS)

if False:
  print('workspace:',workspace,os.path.exists(workspace))
  print('fileName:',fileName,os.path.exists(fileName))
  print('outputPath:',outputPath,os.path.exists(outputPath))

In [ ]:
!ls /content/gdrive/'My Drive'/RPC_TOF/

Aug-200_900_6_830_Aug20th_v1  data    Shift-200_900_6_830_Aug20th_v1
Code			      data_1


In [ ]:
test_form = Waveforms(fileName, train=True)
test_Data = DataLoader(test_form, batch_size=BATCH_SIZE, shuffle=True)
print(test_form[1])

NEvents =  7815.0
{'waveform': tensor([[ 6.6270e-01,  6.3900e-01,  5.9449e-01,  5.3345e-01,  4.6099e-01,
          3.8262e-01,  3.0384e-01,  2.2978e-01,  1.6479e-01,  1.1225e-01,
          7.4304e-02,  5.1869e-02,  4.4597e-02,  5.1012e-02,  6.8694e-02,
          9.4530e-02,  1.2499e-01,  1.5641e-01,  1.8528e-01,  2.0846e-01,
          2.2336e-01,  2.2811e-01,  2.2158e-01,  2.0338e-01,  1.7386e-01,
          1.3396e-01,  8.5143e-02,  2.9230e-02, -3.1701e-02, -9.5442e-02,
         -1.5975e-01, -2.2241e-01, -2.8129e-01, -3.3439e-01, -3.7982e-01,
         -4.1588e-01, -4.4101e-01, -4.5389e-01, -4.5345e-01, -4.3891e-01,
         -4.0993e-01, -3.6659e-01, -3.0951e-01, -2.3985e-01, -1.5936e-01,
         -7.0238e-02,  2.4890e-02,  1.2322e-01,  2.2197e-01,  3.1869e-01,
          4.1162e-01,  4.9996e-01,  5.8429e-01,  6.6675e-01,  7.5132e-01,
          8.4383e-01,  9.5196e-01,  1.0850e+00,  1.2535e+00,  1.4688e+00,
          1.7422e+00,  2.0845e+00,  2.5050e+00,  3.0108e+00,  3.6057e+00,
       

In [ ]:
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

1
Tesla K80


In [ ]:
########################################################################################
## EXCUTION:Training ########
########################################################################################


trainWaveformDataset = Waveforms(fileName, train=True)
testWaveformDataset = Waveforms(fileName, train=False)
trainData = DataLoader(trainWaveformDataset, batch_size=BATCH_SIZE, shuffle=True)
testData = DataLoader(testWaveformDataset, batch_size=BATCH_SIZE, shuffle=False)

comNN = ComNN()
comNN.cuda()
print(comNN)

optimizer = optim.Adam(comNN.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
# https://pytorch.org/docs/stable/optim.html#torch.optim.lr_scheduler.ReduceLROnPlateau
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=LRD)
print(optimizer)
# loss_func = nn.MSELoss().cuda()
loss_func = my_mse_loss

testLoss_his = []
trainLoss_his = []
timeResolution_his = []
mu_his = []
residualTofList = []
predictedTofList = []
labelTofList = []

print("Data: " + DATA_VERSION + "; SEED = " + str(SEED)+"； Epochs = " + str(EPOCHS))
progress = ProgressBar()
for epoch in progress(range(EPOCHS)):
    epoch_loss = 0  # for LR decay rate scheduler
    for index, batch_data in enumerate(trainData):
        input = Variable(batch_data['waveform']).cuda()
        b_y = Variable(batch_data['tof']).cuda().squeeze()
        # print(input.size())
        output = comNN(input).squeeze()
        loss = loss_func(output, b_y, 2)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.data.cpu().numpy()
    scheduler.step(epoch_loss)

    # to store the sigma, mu, loss
    if (epoch) % SAMPLE_EPOCH == 0:
        trainLoss_his.append(loss.data.cpu().numpy())
        residualTofList.clear()
        predictedTofList.clear()
        labelTofList.clear()
        for index, batch_data in enumerate(testData):
            input = Variable(batch_data['waveform']).cuda()
            b_y = Variable(batch_data['tof']).cuda().squeeze()
            output = comNN(input).squeeze()
            loss = loss_func(output, b_y, 2)
            residualTofList.extend(
                (output-b_y).cpu().detach().numpy().tolist())
            predictedTofList.extend(output.cpu().detach().numpy().tolist())
            labelTofList.extend(b_y.cpu().detach().numpy().tolist())
            # print('output length: ',output.cpu().detach().numpy().tolist().__len__())
        # print('toflist length: ',tofList.__len__())
        (mu, sigma) = norm.fit(residualTofList)
        timeResolution_his.append(sigma/math.sqrt(2))
        mu_his.append(mu)
        testLoss_his.append(loss.data.cpu().numpy())


# save model
torch.save(comNN, outputPath+outputPrefix)

NEvents =  7815.0
NEvents =  7815.0


N/A% (0 of 100) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--

ComNN(
  (lstm): LSTM(160, 62, num_layers=4, bias=False, batch_first=True)
  (out1): Linear(in_features=62, out_features=1, bias=False)
)
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0002997938539807655
    weight_decay: 0.0007674443631751687
)
Data: Aug-200_900_6; SEED = 5； Epochs = 100


100% (100 of 100) |######################| Elapsed Time: 0:00:45 Time:  0:00:45


In [ ]:
########################################################################################
## EXCUTION:Plotting ########
########################################################################################
time_start = time.time()
plt.plot(testLoss_his, label='testLoss')
plt.plot(trainLoss_his, label='trainLoss')
plt.grid(True)
# plt.ylim([0.01, 0.08])
# plt.ylabel("MSE [ns^2]")
plt.title("Loss/MSE Vs Epochs")
plt.xlabel("sampled every "+str(SAMPLE_EPOCH)+" epochs")
plt.legend()
plt.savefig(outputPath+outputPrefix+"_Loss.png")
plt.clf()

plt.plot(mu_his)
plt.grid(True)
plt.title("Average predicted ToF Vs Epochs")
plt.ylabel("Average of predicted ToF [ns]")
plt.ylim([-0.1, 0.1])
plt.xlabel("sampled every "+str(SAMPLE_EPOCH)+" epochs")
plt.savefig(outputPath+outputPrefix+"_Mu.png")
plt.clf()

plt.plot(timeResolution_his)
plt.ylabel("Time Resolution [ns]")
plt.grid(True)
plt.ylim([0.05, 0.30])
plt.title("TimeResolution Vs Epochs")
plt.xlabel("sampled every "+str(SAMPLE_EPOCH)+" epochs")
plt.savefig(outputPath+outputPrefix+"_TR.png")
plt.clf()

(mu, sigma) = norm.fit(residualTofList)
n, bins, patches = plt.hist(residualTofList, 100, range=(-1, 1), density=1)
# add a 'best fit' line
y = norm.pdf(bins, mu, sigma)
l = plt.plot(bins, y, 'r--', linewidth=2)
plt.xlabel('Tof residual [ns]')
# plt.ylabel('Probability')
plt.title(r'$\mathrm{Tof\ residual:}\ \mu=%.3f ns,\ \sigma/\sqrt{2}=%.3f ns$' %(mu, sigma/math.sqrt(2)))
plt.grid(True)
plt.savefig(outputPath+outputPrefix+"_TofHist.png")
plt.clf()

for dataVersion in ['300_6','345_6']:
    testDataset(comNN, dataVersion, outputPrefix, input_prefix=workspace+"/data/")

time_end = time.time()
print('time cost', time_end-time_start, 's')

NEvents =  4278.0
Test Data: 300_6
NEvents =  3538.0
Test Data: 345_6
time cost 3.7958462238311768 s


<Figure size 432x288 with 0 Axes>